In [2]:
import numpy as np
import open3d as o3d
import utilities as uti
from plyfile import PlyData, PlyElement

import sys
sys.path.insert(1, './Colors')
import realman_gaussian_color as rgc

curvarute_type = 'gauss'
cam = np.array([0,0,0])
height = 200
width = int(height/2)

data = np.zeros((height, width,2))
data[:,:,0] = None

In [3]:
from PIL import Image
def projectImage(data):
    frame = np.zeros((height, width, 3), dtype=np.uint8)
    frame = frame + 255
    for i in range(height):
        for j in range(width):
            if not np.isnan(data[i][j]):
                color = rgc.getColor(data[i][j])
                frame[i][j] = np.array(color[0:3])
    img = Image.fromarray(frame, 'RGB')
    img.save('./Images/average_gauss_test.png')
    img.show()
    return frame

def computeAverageCurvature(data):
    curvatures = data[:,:,0]
    count = data[:,:,1]
    for i in range(height):
        for j in range(width):
            if not np.isnan(curvatures[i][j]):
                curvatures[i][j] = curvatures[i][j]/count[i][j]
    return curvatures

In [4]:
# Main loop
for i in range(2,3):
    plydata = PlyData.read('./Mesh/results/jean normal/nonback_'+curvarute_type+'_pose_'+str(i)+'.ply')
    points = np.zeros((plydata['vertex'].count,2))
    values = plydata['vertex']['quality']
    points[:,0] = plydata['vertex']['x']
    points[:,1] = plydata['vertex']['y']
    center = [np.average(points[:,0]),np.average(points[:,1])]
    points[:,0] = np.around((points[:,0] - center[0])*width + width/2)
    points[:,1] = np.around((points[:,1] - center[1])*width + width)

    for i in range(len(values)):
        if np.isnan(data[height-int(points[i][1])][width-int(points[i][0])][0]):
            data[height-int(points[i][1])][width-int(points[i][0])][0] = values[i]
        else:
            data[height-int(points[i][1])][width-int(points[i][0])][0] += values[i]
        data[height-int(points[i][1])][width-int(points[i][0])][1] += 1

In [19]:
result = computeAverageCurvature(data)
frame = projectImage(result)

In [31]:
# importing pandas as pd 
import pandas as pd
# Creating the dataframe
print(len(result),len(result[0]))
for i in range(len(result[0])):
    count = [0,0]
    for j in range(len(result)):
        if not np.isnan(result[j][i]):
            count[0] += 1
            count[1] = j
    if count[0] == 1:
        result[count[1] - 1][i]=result[count[1] + 1][i] = result[count[1]][i]
df = pd.DataFrame(data=result)
xframe = df.interpolate(method ='nearest').to_numpy()

200 100


In [32]:
projectImage(xframe)

array([[[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       ...,

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]],

       [[255, 255, 255],
        [255, 255, 255],
        [255, 255, 255],
        ...,
        [255, 255, 255],
        [255, 255, 255],
        [255, 255, 255]]